In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import time
import ipaddress
import datetime
import logging
from collections import Counter
import random
from tqdm import tqdm

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("firewall_simulation.log"),
        logging.StreamHandler()
    ]
)

class NetworkFirewall:
    def __init__(self):
        self.common_ports = {
            20: "FTP Data", 21: "FTP Control", 22: "SSH", 23: "Telnet",
            25: "SMTP", 53: "DNS", 80: "HTTP", 443: "HTTPS",
            3306: "MySQL", 3389: "RDP", 445: "SMB", 1433: "SQL Server",
            8080: "HTTP Alt", 8443: "HTTPS Alt", 5900: "VNC"
        }

        self.blocked_count = 0
        self.allowed_count = 0
        self.suspicious_count = 0
        self.total_packets = 0

        self.blocked_ips = [
            "185.174.102.56", "91.213.8.43", "185.180.12.15",
            "103.43.141.122", "45.95.147.23", "198.12.64.10",
            "5.255.96.50", "185.159.128.78", "194.36.189.51"
        ]

        self.internal_network = ipaddress.IPv4Network("192.168.1.0/24")
        self.dmz_network = ipaddress.IPv4Network("172.16.1.0/24")
        self.sensitive_servers = ["192.168.1.10", "192.168.1.20", "172.16.1.5"]


        self.anomaly_model = None
        self.scaler = StandardScaler()
        self.connection_history = {}
        self.ip_connection_counts = Counter()


        self.alerts = []
        self.suspicious_connections = set()


        self.traffic_logs = []


        self._initialize_anomaly_model()

        logging.info("Network Firewall initialized successfully")

    def _initialize_anomaly_model(self):

        normal_data = self._generate_synthetic_training_data(5000)


        X_scaled = self.scaler.fit_transform(normal_data)


        self.anomaly_model = IsolationForest(
            n_estimators=100,
            max_samples='auto',
            contamination=0.01,
            random_state=42
        )
        self.anomaly_model.fit(X_scaled)

        logging.info("Anomaly detection model initialized and trained")

    def _generate_synthetic_training_data(self, n_samples):

        data = np.zeros((n_samples, 5))

        data[:, 0] = np.random.normal(500, 200, n_samples)


        data[:, 1] = np.random.normal(50, 20, n_samples)


        data[:, 2] = np.random.exponential(60, n_samples)


        data[:, 3] = np.random.normal(2, 0.5, n_samples)


        data[:, 4] = np.random.uniform(0, 24, n_samples) / 24.0

        return data

    def _step1_data_input(self, packet):

        self.total_packets += 1


        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
        logging.debug(f"[STEP 1] Processing packet: {packet['src_ip']}:{packet['src_port']} -> {packet['dst_ip']}:{packet['dst_port']} ({packet['size']} bytes)")


        packet['timestamp'] = timestamp
        packet['packet_id'] = self.total_packets
        self.traffic_logs.append(packet)

        return packet

    def _step2_rule_inspection(self, packet):

        if packet['src_ip'] in self.blocked_ips:
            packet['rule_result'] = "BLOCKED"
            packet['rule_reason'] = "Source IP in blocklist"
            logging.info(f"[STEP 2] BLOCKED packet from {packet['src_ip']} - Source IP in blocklist")
            return packet

        if packet['dst_ip'] in self.sensitive_servers and not self._is_internal_ip(packet['src_ip']):
            if packet['dst_port'] not in [80, 443, 22]:
                packet['rule_result'] = "BLOCKED"
                packet['rule_reason'] = f"Unauthorized access to sensitive server port {packet['dst_port']}"
                logging.info(f"[STEP 2] BLOCKED packet to sensitive server {packet['dst_ip']}:{packet['dst_port']} from external {packet['src_ip']}")
                return packet

        if packet['dst_port'] == 23 and not self._is_internal_ip(packet['src_ip']):
            packet['rule_result'] = "BLOCKED"
            packet['rule_reason'] = "Telnet blocked from external sources"
            logging.info(f"[STEP 2] BLOCKED telnet from external {packet['src_ip']}")
            return packet

        db_ports = [3306, 1433, 5432, 27017]
        if packet['dst_port'] in db_ports and not self._is_internal_ip(packet['src_ip']):
            packet['rule_result'] = "BLOCKED"
            packet['rule_reason'] = f"External access to database port {packet['dst_port']}"
            logging.info(f"[STEP 2] BLOCKED external access to DB port {packet['dst_port']} from {packet['src_ip']}")
            return packet

        self.ip_connection_counts[packet['src_ip']] += 1
        if self.ip_connection_counts[packet['src_ip']] > 500:
            packet['rule_result'] = "SUSPICIOUS"
            packet['rule_reason'] = "High connection rate"
            logging.warning(f"[STEP 2] SUSPICIOUS - High connection rate from {packet['src_ip']}")
            return packet

        packet['rule_result'] = "PASSED"
        packet['rule_reason'] = "Passed rule inspection"
        return packet

    def _step3_anomaly_detection(self, packet):

        if packet['rule_result'] == "BLOCKED":
            return packet

        features = self._extract_features(packet)

        features_scaled = self.scaler.transform(features.reshape(1, -1))


        anomaly_score = self.anomaly_model.decision_function(features_scaled)[0]
        anomaly_prediction = self.anomaly_model.predict(features_scaled)[0]

        packet['anomaly_score'] = anomaly_score


        if anomaly_prediction == -1:
            packet['anomaly_result'] = "ANOMALOUS"
            logging.warning(f"[STEP 3] ANOMALOUS packet detected: {packet['src_ip']}:{packet['src_port']} -> {packet['dst_ip']}:{packet['dst_port']}, Score: {anomaly_score:.4f}")
        else:
            packet['anomaly_result'] = "NORMAL"

        return packet

    def _step4_context_response(self, packet):

        if packet['rule_result'] == "BLOCKED":
            self.blocked_count += 1
            packet['final_action'] = "BLOCKED"
            return packet

        if packet['anomaly_result'] == "ANOMALOUS":
            connection_key = f"{packet['src_ip']}:{packet['dst_ip']}"

            if connection_key in self.suspicious_connections:
                packet['final_action'] = "BLOCKED"
                packet['context_reason'] = "Recurring suspicious activity"
                self.blocked_count += 1
                logging.warning(f"[STEP 4] BLOCKED recurring suspicious activity from {packet['src_ip']}")

                self.alerts.append({
                    'timestamp': packet['timestamp'],
                    'src_ip': packet['src_ip'],
                    'dst_ip': packet['dst_ip'],
                    'alert_type': 'RECURRING_SUSPICIOUS',
                    'score': packet['anomaly_score'],
                    'action': 'BLOCKED'
                })
            else:
                packet['final_action'] = "ALLOWED_WITH_WARNING"
                packet['context_reason'] = "First-time suspicious activity"
                self.suspicious_count += 1
                logging.warning(f"[STEP 4] ALLOWED WITH WARNING first-time suspicious from {packet['src_ip']}")

                self.suspicious_connections.add(connection_key)

                self.alerts.append({
                    'timestamp': packet['timestamp'],
                    'src_ip': packet['src_ip'],
                    'dst_ip': packet['dst_ip'],
                    'alert_type': 'NEW_SUSPICIOUS',
                    'score': packet['anomaly_score'],
                    'action': 'WARNED'
                })

        elif packet['rule_result'] == "SUSPICIOUS":
            packet['final_action'] = "ALLOWED_WITH_WARNING"
            packet['context_reason'] = packet['rule_reason']
            self.suspicious_count += 1
            logging.warning(f"[STEP 4] ALLOWED WITH WARNING - {packet['rule_reason']} from {packet['src_ip']}")

        else:
            packet['final_action'] = "ALLOWED"
            packet['context_reason'] = "Normal traffic"
            self.allowed_count += 1

        return packet

    def _step5_final_output(self, packet):

        action = packet['final_action']

        if action == "BLOCKED":
            message = f"BLOCKED {packet['src_ip']}:{packet['src_port']} -> {packet['dst_ip']}:{packet['dst_port']}"
            if 'context_reason' in packet:
                message += f" Reason: {packet['context_reason']}"
            elif 'rule_reason' in packet:
                message += f" Reason: {packet['rule_reason']}"
            logging.info(f"[STEP 5] {message}")

        elif action == "ALLOWED_WITH_WARNING":
            message = f"WARNED {packet['src_ip']}:{packet['src_port']} -> {packet['dst_ip']}:{packet['dst_port']}"
            if 'context_reason' in packet:
                message += f" Reason: {packet['context_reason']}"
            logging.info(f"[STEP 5] {message}")

        elif action == "ALLOWED":
            logging.debug(f"[STEP 5] ALLOWED {packet['src_ip']}:{packet['src_port']} -> {packet['dst_ip']}:{packet['dst_port']}")

        return packet

    def process_packet(self, packet):

        packet = self._step1_data_input(packet)

        packet = self._step2_rule_inspection(packet)

        packet = self._step3_anomaly_detection(packet)

        packet = self._step4_context_response(packet)

        packet = self._step5_final_output(packet)

        return packet

    def _is_internal_ip(self, ip):
        try:
            return ipaddress.IPv4Address(ip) in self.internal_network or ipaddress.IPv4Address(ip) in self.dmz_network
        except:
            return False

    def _extract_features(self, packet):

        hour = datetime.datetime.now().hour / 24.0

        src_ip = packet['src_ip']
        if src_ip not in self.connection_history:
            self.connection_history[src_ip] = {
                'last_seen': time.time(),
                'packet_sizes': [packet['size']],
                'ports_accessed': [packet['dst_port']]
            }
            packet_rate = 0.1
            connection_duration = 0.1
            port_entropy = 0.0
        else:
            history = self.connection_history[src_ip]
            current_time = time.time()


            time_diff = current_time - history['last_seen']
            history['last_seen'] = current_time

            history['packet_sizes'].append(packet['size'])
            if len(history['packet_sizes']) > 100:
                history['packet_sizes'] = history['packet_sizes'][-100:]


            history['ports_accessed'].append(packet['dst_port'])
            if len(history['ports_accessed']) > 100:
                history['ports_accessed'] = history['ports_accessed'][-100:]

            packet_rate = 1.0 / max(time_diff, 0.1)

            connection_duration = 10.0

            port_counts = Counter(history['ports_accessed'])
            total_ports = len(history['ports_accessed'])
            port_entropy = 0.0
            for port, count in port_counts.items():
                p = count / total_ports
                port_entropy -= p * np.log2(p)


        return np.array([
            packet['size'],
            packet_rate,
            connection_duration,
            port_entropy,
            hour
        ])

    def generate_reports(self):
        stats = {
            'total_packets': self.total_packets,
            'allowed': self.allowed_count,
            'blocked': self.blocked_count,
            'suspicious': self.suspicious_count,
            'alert_count': len(self.alerts),
            'unique_sources': len(set(log['src_ip'] for log in self.traffic_logs)),
            'unique_destinations': len(set(log['dst_ip'] for log in self.traffic_logs))
        }


        if self.total_packets > 0:
            stats['allowed_percentage'] = (self.allowed_count / self.total_packets) * 100
            stats['blocked_percentage'] = (self.blocked_count / self.total_packets) * 100
            stats['suspicious_percentage'] = (self.suspicious_count / self.total_packets) * 100
        else:
            stats['allowed_percentage'] = 0
            stats['blocked_percentage'] = 0
            stats['suspicious_percentage'] = 0


        src_counts = Counter(log['src_ip'] for log in self.traffic_logs)
        dst_counts = Counter(log['dst_ip'] for log in self.traffic_logs)
        port_counts = Counter(log['dst_port'] for log in self.traffic_logs)

        stats['top_sources'] = src_counts.most_common(5)
        stats['top_destinations'] = dst_counts.most_common(5)
        stats['top_ports'] = [(port, count, self.common_ports.get(port, "Unknown"))
                            for port, count in port_counts.most_common(5)]


        logs_df = pd.DataFrame(self.traffic_logs)


        return stats, logs_df

    def plot_traffic_summary(self, logs_df):

        plt.figure(figsize=(15, 10))


        plt.subplot(2, 2, 1)
        action_counts = logs_df['final_action'].value_counts()
        plt.pie(action_counts, labels=action_counts.index, autopct='%1.1f%%',
                colors=['green', 'red', 'orange'] if 'ALLOWED_WITH_WARNING' in action_counts.index else ['green', 'red'])
        plt.title('Traffic Action Distribution')


        plt.subplot(2, 2, 2)
        top_sources = logs_df['src_ip'].value_counts().head(10)
        top_sources.plot(kind='barh')
        plt.title('Top 10 Source IPs')
        plt.xlabel('Packet Count')
        plt.ylabel('Source IP')
        plt.tight_layout()


        plt.subplot(2, 2, 3)
        top_ports = logs_df['dst_port'].value_counts().head(10)
        top_ports.plot(kind='barh')
        plt.title('Top 10 Destination Ports')
        plt.xlabel('Packet Count')
        plt.ylabel('Destination Port')


        plt.subplot(2, 2, 4)
        if 'anomaly_score' in logs_df.columns:
            sns.histplot(logs_df['anomaly_score'].dropna(), kde=True)
            plt.title('Anomaly Score Distribution')
            plt.xlabel('Anomaly Score')
            plt.ylabel('Frequency')
        else:
            plt.text(0.5, 0.5, 'No anomaly scores available',
                     horizontalalignment='center', verticalalignment='center')
            plt.title('Anomaly Score Distribution')

        plt.tight_layout()
        plt.savefig('firewall_traffic_summary.png')
        plt.close()


        if 'timestamp' in logs_df.columns:
            try:
                logs_df['timestamp'] = pd.to_datetime(logs_df['timestamp'])
                logs_df.set_index('timestamp', inplace=True)


                traffic_by_time = logs_df.resample('1min').size()

                plt.figure(figsize=(12, 6))
                traffic_by_time.plot()
                plt.title('Traffic Volume Over Time')
                plt.xlabel('Time')
                plt.ylabel('Packet Count')
                plt.tight_layout()
                plt.savefig('traffic_over_time.png')
                plt.close()
            except:
                logging.warning("Could not generate time-based traffic plot")

In [3]:
def generate_synthetic_packet():

    internal_ips = [f"192.168.1.{i}" for i in range(1, 50)]
    external_ips = [
        "203.0.113.45", "198.51.100.67", "8.8.8.8", "1.1.1.1",
        "185.174.102.56", "91.213.8.43", "45.95.147.23", "65.21.54.1",
        "104.21.31.84", "172.67.169.39", "140.82.121.4", "35.227.248.234",
        "185.180.12.15", "103.43.141.122", "194.36.189.51", "5.255.96.50"
    ]

    dmz_ips = [f"172.16.1.{i}" for i in range(1, 10)]

    common_ports = [21, 22, 23, 25, 53, 80, 443, 3306, 3389, 445, 8080, 8443]


    direction_roll = random.random()

    if direction_roll < 0.4:
        src_ip = random.choice(internal_ips)
        dst_ip = random.choice(external_ips)
    elif direction_roll < 0.7:
        src_ip = random.choice(external_ips)
        dst_ip = random.choice(internal_ips + dmz_ips)
    elif direction_roll < 0.9:
        src_ip = random.choice(internal_ips)
        dst_ip = random.choice(internal_ips + dmz_ips)
    else:
        src_ip = random.choice(dmz_ips)
        dst_ip = random.choice(internal_ips + external_ips)


    is_attack = random.random() < 0.05

    if is_attack:

        packet_size = random.randint(1500, 65000)


        if random.random() < 0.5:
            dst_port = random.randint(1024, 65535)
        else:
            dst_port = random.choice([3306, 1433, 5432, 23, 22])

        src_port = random.randint(10000, 65000)

    else:

        packet_size = random.randint(64, 1460)
        dst_port = random.choice(common_ports)
        src_port = random.randint(10000, 65000)


    packet = {
        'src_ip': src_ip,
        'src_port': src_port,
        'dst_ip': dst_ip,
        'dst_port': dst_port,
        'size': packet_size,
        'protocol': random.choice(['TCP', 'UDP', 'ICMP']),
        'flags': random.choice(['SYN', 'ACK', 'SYN-ACK', 'FIN', 'RST', 'NONE']),
    }

    return packet

def run_simulation(duration_seconds=60, packets_per_second=10):

    firewall = NetworkFirewall()

    total_packets = duration_seconds * packets_per_second
    start_time = time.time()

    logging.info(f"Starting simulation - Processing {total_packets} packets over {duration_seconds} seconds")

    try:
        for _ in tqdm(range(total_packets), desc="Processing packets"):

            packet = generate_synthetic_packet()

            firewall.process_packet(packet)

            sleep_time = max(0, random.normalvariate(1.0 / packets_per_second, 0.01))
            time.sleep(sleep_time)

    except KeyboardInterrupt:
        logging.info("Simulation interrupted by user")

    end_time = time.time()
    actual_duration = end_time - start_time
    actual_rate = firewall.total_packets / actual_duration

    logging.info(f"Simulation complete - Processed {firewall.total_packets} packets in {actual_duration:.2f} seconds ({actual_rate:.2f} packets/sec)")

    stats, logs_df = firewall.generate_reports()

    firewall.plot_traffic_summary(logs_df)

    return firewall, stats, logs_df

In [4]:
def display_simulation_results(stats, logs_df):

    print("\n" + "="*80)
    print(" "*30 + "SIMULATION RESULTS")
    print("="*80)


    print("\n[TRAFFIC STATISTICS]")
    print(f"Total Packets Processed: {stats['total_packets']}")
    print(f"Allowed Packets: {stats['allowed']} ({stats['allowed_percentage']:.2f}%)")
    print(f"Blocked Packets: {stats['blocked']} ({stats['blocked_percentage']:.2f}%)")
    print(f"Suspicious Packets: {stats['suspicious']} ({stats['suspicious_percentage']:.2f}%)")
    print(f"Unique Source IPs: {stats['unique_sources']}")
    print(f"Unique Destination IPs: {stats['unique_destinations']}")
    print(f"Total Security Alerts: {stats['alert_count']}")

    print("\n[TOP 5 TRAFFIC SOURCES]")
    for ip, count in stats['top_sources']:
        print(f"{ip}: {count} packets ({(count/stats['total_packets'])*100:.2f}%)")


    print("\n[TOP 5 TRAFFIC DESTINATIONS]")
    for ip, count in stats['top_destinations']:
        print(f"{ip}: {count} packets ({(count/stats['total_packets'])*100:.2f}%)")


    print("\n[TOP 5 DESTINATION PORTS]")
    for port, count, service in stats['top_ports']:
        print(f"Port {port} ({service}): {count} packets ({(count/stats['total_packets'])*100:.2f}%)")


    print("\n[PACKET ACTION DISTRIBUTION]")
    action_counts = logs_df['final_action'].value_counts()
    for action, count in action_counts.items():
        print(f"{action}: {count} packets ({(count/stats['total_packets'])*100:.2f}%)")


    if 'anomaly_score' in logs_df.columns:
        print("\n[ANOMALY SCORE STATISTICS]")
        anomaly_stats = logs_df['anomaly_score'].describe()
        print(f"Mean Anomaly Score: {anomaly_stats['mean']:.4f}")
        print(f"Min Anomaly Score: {anomaly_stats['min']:.4f}")
        print(f"Max Anomaly Score: {anomaly_stats['max']:.4f}")
        print(f"25th Percentile: {anomaly_stats['25%']:.4f}")
        print(f"Median Anomaly Score: {anomaly_stats['50%']:.4f}")
        print(f"75th Percentile: {anomaly_stats['75%']:.4f}")

    print("\n" + "="*80)
    print("Visualization files created:")
    print("- firewall_traffic_summary.png - Summary of traffic patterns")
    print("- traffic_over_time.png - Traffic volume over time")
    print("="*80 + "\n")


if __name__ == "__main__":
    print("\n" + "="*80)
    print(" "*25 + "NETWORK FIREWALL SIMULATION")
    print("="*80)
    print("\nSimulation Setup:")
    print("- Framework: Custom Python implementation")
    print("- Components: Data Input -> Rule Inspection -> Anomaly Detection -> Context Response -> Final Output")
    print("- Attack Simulation: 5% of traffic is attack/anomalous")
    print("- Real IP Addresses: Using realistic internal, DMZ, and external IP ranges")
    print("- ML Algorithm: Isolation Forest for anomaly detection")
    print("- Duration: 60 seconds")
    print("- Traffic Rate: 10 packets per second")
    print("\nStarting simulation...")


    firewall, stats, logs_df = run_simulation(duration_seconds=60, packets_per_second=10)

    display_simulation_results(stats, logs_df)


                         NETWORK FIREWALL SIMULATION

Simulation Setup:
- Framework: Custom Python implementation
- Components: Data Input -> Rule Inspection -> Anomaly Detection -> Context Response -> Final Output
- Attack Simulation: 5% of traffic is attack/anomalous
- Real IP Addresses: Using realistic internal, DMZ, and external IP ranges
- ML Algorithm: Isolation Forest for anomaly detection
- Duration: 60 seconds
- Traffic Rate: 10 packets per second

Starting simulation...


Processing packets: 100%|██████████| 600/600 [01:16<00:00,  7.83it/s]



                              SIMULATION RESULTS

[TRAFFIC STATISTICS]
Total Packets Processed: 600
Allowed Packets: 352 (58.67%)
Blocked Packets: 117 (19.50%)
Suspicious Packets: 131 (21.83%)
Unique Source IPs: 74
Unique Destination IPs: 74
Total Security Alerts: 133

[TOP 5 TRAFFIC SOURCES]
185.174.102.56: 17 packets (2.83%)
91.213.8.43: 17 packets (2.83%)
192.168.1.49: 17 packets (2.83%)
45.95.147.23: 15 packets (2.50%)
194.36.189.51: 15 packets (2.50%)

[TOP 5 TRAFFIC DESTINATIONS]
203.0.113.45: 24 packets (4.00%)
35.227.248.234: 20 packets (3.33%)
103.43.141.122: 18 packets (3.00%)
8.8.8.8: 17 packets (2.83%)
65.21.54.1: 16 packets (2.67%)

[TOP 5 DESTINATION PORTS]
Port 443 (HTTPS): 57 packets (9.50%)
Port 3306 (MySQL): 54 packets (9.00%)
Port 8443 (HTTPS Alt): 53 packets (8.83%)
Port 53 (DNS): 53 packets (8.83%)
Port 23 (Telnet): 52 packets (8.67%)

[PACKET ACTION DISTRIBUTION]
ALLOWED: 352 packets (58.67%)
BLOCKED: 117 packets (19.50%)

[ANOMALY SCORE STATISTICS]
Mean Anomaly 

In [ ]:
def test_ip_addresses(firewall, test_ips, num_packets_per_ip=10):

    results = []

    for ip in test_ips:
        logging.info(f"Testing IP: {ip} with {num_packets_per_ip} packets")
        ip_results = {'src_ip': ip, 'packets': []}

        for _ in range(num_packets_per_ip):

            packet = generate_synthetic_packet()
            packet['src_ip'] = ip


            processed_packet = firewall.process_packet(packet)


            packet_info = {
                'dst_ip': processed_packet['dst_ip'],
                'dst_port': processed_packet['dst_port'],
                'size': processed_packet['size'],
                'final_action': processed_packet['final_action'],
                'anomaly_score': processed_packet.get('anomaly_score', 0),
                'rule_reason': processed_packet.get('rule_reason', ''),
                'context_reason': processed_packet.get('context_reason', '')
            }
            ip_results['packets'].append(packet_info)

        results.append(ip_results)


    print("\n" + "="*80)
    print(" "*30 + "IP TEST RESULTS")
    print("="*80)

    for ip_result in results:
        ip = ip_result['src_ip']
        print(f"\nResults for IP: {ip}")
        print("-"*40)

        action_counts = Counter(p['final_action'] for p in ip_result['packets'])
        for action, count in action_counts.items():
            print(f"{action}: {count} packets ({(count/num_packets_per_ip)*100:.2f}%)")

        avg_anomaly_score = np.mean([p['anomaly_score'] for p in ip_result['packets']])
        print(f"Average Anomaly Score: {avg_anomaly_score:.4f}")

        blocked_packets = [p for p in ip_result['packets'] if p['final_action'] == 'BLOCKED']
        if blocked_packets:
            print("\nBlocked Packets:")
            for p in blocked_packets:
                print(f"  To {p['dst_ip']}:{p['dst_port']} (Size: {p['size']}) - "
                      f"Reason: {p['rule_reason'] or p['context_reason']}")

        suspicious_packets = [p for p in ip_result['packets'] if p['final_action'] == 'ALLOWED_WITH_WARNING']
        if suspicious_packets:
            print("\nSuspicious Packets:")
            for p in suspicious_packets:
                print(f"  To {p['dst_ip']}:{p['dst_port']} (Size: {p['size']}) - "
                      f"Reason: {p['context_reason']}, Anomaly Score: {p['anomaly_score']:.4f}")

    print("\n" + "="*80)

    return results

if __name__ == "__main__":
    firewall = NetworkFirewall()

    test_ips = [
        "203.0.113.45",
        "192.168.1.100",
        "185.174.102.56",
        "172.16.1.5"
    ]

    print("\nRunning initial simulation...")
    firewall, stats, logs_df = run_simulation(duration_seconds=30, packets_per_second=10)

    print("\nTesting specific IP addresses...")
    test_results = test_ip_addresses(firewall, test_ips, num_packets_per_ip=10)



Running initial simulation...


Processing packets: 100%|██████████| 300/300 [00:37<00:00,  8.06it/s]



Testing specific IP addresses...

                              IP TEST RESULTS

Results for IP: 203.0.113.45
----------------------------------------
ALLOWED: 5 packets (50.00%)
BLOCKED: 1 packets (10.00%)
Average Anomaly Score: 0.0209

Blocked Packets:
  To 172.16.1.1:3306 (Size: 437) - Reason: External access to database port 3306

Suspicious Packets:
  To 192.168.1.23:3389 (Size: 1124) - Reason: First-time suspicious activity, Anomaly Score: -0.0063
  To 192.168.1.47:53 (Size: 1389) - Reason: First-time suspicious activity, Anomaly Score: -0.0026
  To 192.168.1.35:22 (Size: 1385) - Reason: First-time suspicious activity, Anomaly Score: -0.0038
  To 192.168.1.48:8443 (Size: 1397) - Reason: First-time suspicious activity, Anomaly Score: -0.0038

Results for IP: 192.168.1.100
----------------------------------------
ALLOWED: 8 packets (80.00%)
Average Anomaly Score: 0.0386

Suspicious Packets:
  To 192.168.1.15:8080 (Size: 926) - Reason: First-time suspicious activity, Anomaly Score: